# Classical Model

In [1]:
import re
import requests
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams
from collections import defaultdict
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split

In [2]:
r = requests.get("https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt")
data = r.text

train_data=data[:723135]
#80%

test_data=data[723135:]
#20%

In [3]:
def data_cleaner(data):
	f=data.replace(u'\xa0', u' ')
	f=re.sub(r'\n', ' ', f)
	f=re.sub(r'\r', ' ', f)
	l = sent_tokenize(f)
	l=list(filter(None, l))
	return l
data_test=[]
final1=data_cleaner(test_data)
for i in final1:
    s = i.lower()
    s = re.sub(r"[^a-zA-Z0-9'\s]", ' ', s)
    s='<s> '+s+' </s>'
    data_test.append(s)


In [4]:
def counter(n_gram):
	d=dict()
	for i in n_gram:
		if i in d:
			d[i]+=1
		else:
			d[i]=1
	return(d)

In [5]:
def prob_finder(n1_gram,n1_gram_count,n0_gram_count):
	# Along with Add-1 smoothing
	le=len(n1_gram_count)
	total_n_grams=len(n1_gram)
	unique_grams=list(n1_gram_count.keys())
	prob_vec=[0 for i in range(le)]
	if len(n1_gram[0])==1:
		for i in range(le):
			prob_vec[i]=(n1_gram_count[unique_grams[i]])/total_n_grams
	elif len(n1_gram[0])==2:
		for i in range(le):
			prob_vec[i]=n1_gram_count[unique_grams[i]]/n0_gram_count[(unique_grams[i][0],)]
	elif len(n1_gram[0])==3:
		for i in range(le):
			prob_vec[i]=n1_gram_count[unique_grams[i]]/n0_gram_count[(unique_grams[i][0],unique_grams[i][1])]
	elif len(n1_gram[0])==4:
		for i in range(le):
			prob_vec[i]=n1_gram_count[unique_grams[i]]/n0_gram_count[(unique_grams[i][0],unique_grams[i][1],unique_grams[i][2])]
	return(unique_grams,prob_vec)

In [6]:
final_data=data_cleaner(train_data)
unigrams=[]
bigrams=[]
trigrams=[]
quadgrams=[]
data_train=[]
for i in final_data:
    s = i.lower()
    s = re.sub(r"[^a-zA-Z0-9'\s]", ' ', s)
    s='<s> '+s+' </s>'
    data_train.append(s)
    tokens = [token for token in s.split(" ") if token != ""]
    unigrams+=list(ngrams(tokens, 1))
    bigrams+=list(ngrams(tokens, 2))
    trigrams+=list(ngrams(tokens, 3))
    quadgrams+=list(ngrams(tokens, 4))
unigram_count=counter(unigrams)
bigram_count=counter(bigrams)
trigram_count=counter(trigrams)
quadgram_count=counter(quadgrams)

In [7]:
uni_prob=prob_finder(unigrams,unigram_count,[])
bi_prob=prob_finder(bigrams,bigram_count,unigram_count)
tri_prob=prob_finder(trigrams,trigram_count,bigram_count)
quad_prob=prob_finder(quadgrams,quadgram_count,trigram_count)
# each of these is a list --> [all_unique_n_grams,corresponding_probabilities] 
req_strings=[]
def unigram_generator(uni_prob):
# unigram sentence generator
	print("5 sentences by unigram")
	print()
	for i in range(5):
		s1out=''
		while '</s>' not in s1out :
			rand=np.random.multinomial(15,uni_prob[1])
			ma=np.argmax(rand)
			s1out+=uni_prob[0][ma][0]+' '
		s1out=s1out.replace('<s> ','')
		print(s1out.replace(' </s>','.'))
	print()
	print()
	return(1)


def bigram_generator(bi_prob):
	print("5 sentences by bigram")
	print()

	# bigram sentence generator
	for m in range(5):
		s2out=''
		start='<s>'
		while '</s>' not in s2out :
			reqlist=[]
			req_probs=[]
			for i in range(len(bi_prob[0])):
				if bi_prob[0][i][0]==start:
					reqlist.append(bi_prob[0][i])
					req_probs.append(bi_prob[1][i])
			su=sum(req_probs)
			for i in range(len(req_probs)):
				req_probs[i]=req_probs[i]/su
			rand=np.random.multinomial(5,req_probs)
			ma=np.argmax(rand)
			s2out+=reqlist[ma][1]+' '

			start=reqlist[ma][1]
		s2out=s2out.replace(' </s>','.')
		print(s2out)
	print()
	print()

def trigram_generator(tro_prob):
	print("5 sentences by trigram")
	print()
	# trigram sentence generator
	for m in range(5):
		s2out=''
		start=['<s>','<s>']
		c=0
		while '</s>' not in s2out :
			reqlist=[]
			req_probs=[]
			for i in range(len(tri_prob[0])):
				if ((c>0 and (tri_prob[0][i][0]==start[0] and tri_prob[0][i][1]==start[1])) or (c==0 and tri_prob[0][i][0]==start[0])):
					reqlist.append(tri_prob[0][i])
					req_probs.append(tri_prob[1][i])
			su=sum(req_probs)
			for i in range(len(req_probs)):
				req_probs[i]=req_probs[i]/su
			rand=np.random.multinomial(5,req_probs)
			ma=np.argmax(rand)
			if c>0:
				s2out+=reqlist[ma][2]+' '
				start=[start[1],reqlist[ma][2]]
			else:
				s2out+=reqlist[ma][1]+' '
				start=[start[1],reqlist[ma][1]]
			c+=1
		s2out=s2out.replace(' </s>','.')
		print(s2out)
	print()
	print()

def quadgram_generator(quad_prob):
	print("5 sentences by quadgram")
	print()

	# bigram sentence generator
	for m in range(5):
		s2out=''
		start=['<s>','<s>','<s>']
		c=0
		while '</s>' not in s2out :
			reqlist=[]
			req_probs=[]
			for i in range(len(quad_prob[0])):
#				print(start,quad_prob[0][i][:3],start==quad_prob[0][i][:3])
				if ((c==0 and quad_prob[0][i][0]==start[0]) or (c>0 and [quad_prob[0][i][0],quad_prob[0][i][1],quad_prob[0][i][2]]==start)):
					reqlist.append(quad_prob[0][i])
					req_probs.append(quad_prob[1][i])
			su=sum(req_probs)
			for i in range(len(req_probs)):
				req_probs[i]=req_probs[i]/su
			rand=np.random.multinomial(5,req_probs)
			ma=np.argmax(rand)

			if c>0:
				s2out+=reqlist[ma][3]+' '
#				print(start)
#				print(reqlist[ma])
				start=[start[1],start[2],reqlist[ma][3]]
			else:
				s2out+=reqlist[ma][1]+' '+reqlist[ma][2]+' '+reqlist[ma][3]+' '
				start=[reqlist[ma][1],reqlist[ma][2],reqlist[ma][3]]
			c+=1
		s2out=s2out.replace(' </s>','.')
		print(s2out)
	print()
	print()

# here n is 1|2|3|4 based on which gram should be used to generate sentences.
def generator(n):
	if n==1:
		unigram_generator(uni_prob)
	elif n==2:
		bigram_generator(bi_prob)
	elif n==3:
		trigram_generator(tri_prob)
	elif n==4:
		quadgram_generator(quad_prob)
	return 1
#_____________________________________________________________________________________________________


# call the function here
generator(1)
generator(2)
generator(3)
generator(4)


5 sentences by unigram

and that the great. 
you so he. 
the. 
</s> 
</s> 


5 sentences by bigram

so unfair. 
i could be a real estate. 
we re doing this and i m going to be allowed to tell you hear a big fat ugly bubble that are so it. 
i have to make the way it. 
i love mexico. 


5 sentences by trigram

jerry falwell jr endorsed me. 
stay there go in and get them to come in and get him back. 
you know what you ve got to be an election he should have gotten if i lose i ll tell you. 
politicians are all talk no action. 
i'm not getting millions and millions of dollars from all these other guys they were forced to make our country. 


5 sentences by quadgram

finally america no longer has a clear understanding of our foreign policy goals. 
it' can't be mitt because mitt ran and failed. 
but you know what. 
and it s been a very exciting process. 
my patients are beside themselves. 




1

In [8]:

def smoothing(test_n_gram,n1_gram,n1_gram_count,n0_gram_count):
	# Along with Add-1 smoothing
	le=len(n1_gram_count)
	total_n_grams=len(n1_gram)
	unique_grams=list(n1_gram_count.keys())
	prob_vec=[0 for i in range(le)]
	d={}
	for i in test_n_gram:
		if i not in d:
			d[i]=0
	extra=0
	for i in test_n_gram:
		if i not in n1_gram_count:
			extra+=1

	if len(n1_gram[0])==1:
		for i in range(le):
			if unique_grams[i] in d:
				d[unique_grams[i]]=(n1_gram_count[unique_grams[i]])/((total_n_grams+extra)*10)
	elif len(n1_gram[0])==2:
		for i in range(le):
			if unique_grams[i] in d:
				d[unique_grams[i]]=(n1_gram_count[unique_grams[i]])/(n0_gram_count[(unique_grams[i][0],)])
	elif len(n1_gram[0])==3:
		for i in range(le):
			if unique_grams[i] in d:
				d[unique_grams[i]]=(1+n1_gram_count[unique_grams[i]])/(le+n0_gram_count[(unique_grams[i][0],unique_grams[i][1])])
	elif len(n1_gram[0])==4:
		for i in range(le):
			if unique_grams[i] in d:
				d[unique_grams[i]]=(1+n1_gram_count[unique_grams[i]])/(le+n0_gram_count[(unique_grams[i][0],unique_grams[i][1],unique_grams[i][2])])
	for i in d:

		if (len(i)==1 and d[i]==0):
			d[i]=.1/(total_n_grams+extra)
		elif (len(i)==2 and d[i]==0):
			d[i]=min(bi_prob[1])
		elif (len(i)==3 and d[i]==0):
			d[i]=min(tri_prob[1])
		else:
			d[i]=min(quad_prob[1])
	return(d)


#_____________________________________________________________________________________________________
def smoothing_2(test_data):
	final_data=data_cleaner(test_data)
	test_unigrams=[]
	test_bigrams=[]
	test_trigrams=[]
	test_quadgrams=[]
	for i in final_data:
		s = i.lower()
		s = re.sub(r"[^a-zA-Z0-9'\s]", ' ', s)
		s='<s> '+s+' </s>'
		tokens = [token for token in s.split(" ") if token != ""]
		test_unigrams+=list(ngrams(tokens, 1))
		test_bigrams+=list(ngrams(tokens, 2))
		test_trigrams+=list(ngrams(tokens, 3))
		test_quadgrams+=list(ngrams(tokens, 4))
	k1=smoothing(test_unigrams,unigrams,unigram_count,unigram_count)
	k2=smoothing(test_bigrams,bigrams,bigram_count,unigram_count)
	k3=smoothing(test_trigrams,trigrams,trigram_count,bigram_count)
	k4=smoothing(test_quadgrams,quadgrams,quadgram_count,trigram_count)
	return (k1,k2,k3,k4)
def perplexity(k):    
	p=1
	s=0
	total_len=len(k)
	for i in k:
		s+=k[i]
	for i in k:
		k[i]=5*k[i]
		p=p*((k[i])**(1/total_len))
	perp=(1/p)
	return(perp)

print("Unigram model perplexity : " + str(perplexity(smoothing_2(test_data)[0])))
print("Bigram model perplexity : " + str(perplexity(smoothing_2(test_data)[1])))
print("Trigram model perplexity : " + str(perplexity(smoothing_2(test_data)[2])))
print("Quadgram model perplexity : " + str(perplexity(smoothing_2(test_data)[3])))
# this calculation would take time around 5 min

Unigram model perplexity : 884.6108602059542
Bigram model perplexity : 583.5676393384372
Trigram model perplexity : 281.6499115272768
Quadgram model perplexity : 137.79999999998728


In [21]:
tokenizer = Tokenizer(num_words=None, filters=[], lower=True, split=" ") # data is already filtered, so no need to filter any thing else
tokenizer.fit_on_texts(data_train) # fitting indices to words
word_index = tokenizer.word_index # word -> index
index_word = {} # index -> word
for word in word_index:
    index_word[word_index[word]] = word

vocab_size = len(tokenizer.word_index) # Vocabulary size
print('Vocabulary Size:', vocab_size)
data_train = tokenizer.texts_to_sequences(data_train)

Vocabulary Size: 5242


# Neural Model

In [26]:
class neural_model:
    def __init__(self, data, tokenizer, word_index, index_word, max_sequence_len=15, hiddenlayer="Vanilla RNN", epochs=20, batch_size=256):
        self.data= data
        self.tokenizer = tokenizer
        self.sentences = data
        self.wordtoindex = word_index
        self.indextoword = index_word
        self.max_sequence_len = max_sequence_len
        self.vocab = len(self.wordtoindex) + 1
        self.hidden = hiddenlayer
        self.epochs = epochs
        self.batch_size = batch_size
    
    def train(self):
        self.prepare_data()
        self.model = self.model_create()
        
    def evaluate(self, data):
        data1 = self.tokenizer.texts_to_sequences(data)
        input_sequences = []
        for line in data1:
            for i in range(1, len(line)):
                n_gram_sequence = line[:i+1]
                input_sequences.append(n_gram_sequence)
        input_sequences = np.array(pad_sequences(input_sequences,maxlen = self.max_sequence_len, padding='pre'))
        X, y = input_sequences[:,:-1],input_sequences[:,-1]
        y = to_categorical(y, num_classes=self.vocab)
        loss = self.model.evaluate(X,y, verbose=0)
        return np.exp(loss)
        
        
    def prepare_data(self):
        input_sequences = []
        for line in self.data:
            for i in range(1, len(line)):
                n_gram_sequence = line[:i+1]
                input_sequences.append(n_gram_sequence)
        input_sequences = np.array(pad_sequences(input_sequences,maxlen = self.max_sequence_len, padding='pre'))
        self.X, y = input_sequences[:,:-1],input_sequences[:,-1]
        self.y = to_categorical(y, num_classes=self.vocab)

    def model_create(self):
        input_len = self.max_sequence_len - 1
        model = Sequential()
        model.add(Embedding(self.vocab, 128, input_length=input_len))
        if self.hidden == "Vanilla RNN":
            model.add(SimpleRNN(256))
        elif self.hidden == "LSTM":
            model.add(LSTM(256))
        model.add(Dropout(0.2))
        model.add(Dense(self.vocab, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        print(model.summary())
        model.fit(self.X, self.y, epochs=self.epochs, verbose=1, batch_size=self.batch_size)
        return model
        
    def generate_text(self, n_sent):
        in_text, result = ["<s>"], ""
        sent_len=1
        started = 0
        while (n_sent>0):
            encoded = np.array(self.tokenizer.texts_to_sequences([in_text])[0])
            encoded = pad_sequences([encoded], maxlen= self.max_sequence_len-1, padding='pre')
            probs = list(np.transpose(self.model.predict_proba(encoded, verbose=0))[:,0])
            a=sum(probs)*1.1
            yhat=[i/a for i in probs]
            out = list(np.random.multinomial(2,yhat))
            i = out.index(max(out))
            out_word = self.indextoword[i]
            if out_word=="</s>" or sent_len == self.max_sequence_len:
                out_word = "."
                in_text = ["<s>"]
                n_sent-=1
                sent_len=1
                result = result + out_word
                started=0
            else:
                in_text+=[out_word]
                sent_len+=1
                if not started:
                    started = 1
                    out_word = out_word.capitalize()
                result = result + " " + out_word
        return result.strip()

In [27]:

model = neural_model(data=data_train, tokenizer=tokenizer, word_index=word_index, index_word=index_word, max_sequence_len=15, hiddenlayer="Vanilla RNN", epochs=10, batch_size=256)
model.train()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 14, 128)           671104    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               98560     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5243)              1347451   
Total params: 2,117,115
Trainable params: 2,117,115
Non-trainable params: 0
_________________________________________________________________
None
Train on 153570 samples
Epoch 1/10
153570/153570 [==============================] - 14s 90us/sample - loss: 5.5566
Epoch 2/10
153570/153570 [==============================] - 12s 80us/sample - loss: 4.6090s - loss:
Epoch 3/10
153570/1535

In [28]:
model_lstm = neural_model(data=data_train, tokenizer=tokenizer, word_index=word_index, index_word=index_word, max_sequence_len=15, hiddenlayer="LSTM", epochs=10, batch_size=256)
model_lstm.train()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 14, 128)           671104    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5243)              1347451   
Total params: 2,412,795
Trainable params: 2,412,795
Non-trainable params: 0
_________________________________________________________________
None
Train on 153570 samples
Epoch 1/10
153570/153570 [==============================] - 14s 92us/sample - loss: 5.5799
Epoch 2/10
153570/153570 [==============================] - 13s 86us/sample - loss: 4.7835
Epoch 3/10
153570/153570 [=====

In [29]:
print("Vanilla RNN model\n\n")
print(model.generate_text(5))
print("\n\nLSTM model\n\n")
print(model_lstm.generate_text(5))

Vanilla RNN model


You have to you have to know mr trump they re in a bridge. And i have a great smaller country on the world. And we re going to come in legally. We have four or with the veterans that add to the republicans. They have polls they need people at the united states.


LSTM model


Thank you something. I will tell you. The highest tour we will just i have iran to be america to have. I think he s a fantastic negotiator. I said we re bringing a poll.


In [30]:
print("Vanilla RNN model:")
print(model.evaluate(data_test))
print("\nLSTM model:")
print(model_lstm.evaluate(data_test))

Vanilla RNN model:
88.60448136480957

LSTM model:
83.86604432397661


Both The Neural Approach and Natural language model approach generate good senteces except the unigram one, based on the perplexities neural dominates the other because of taking into consideration more that 4 words in a window and learning.